1.无重复插入neo4j，在插入过程中异步处理以下事情：

处理实体id文件，如果数据库中已存在，则不处理，否则a+添加到id文件中
处理triple文件，a+追加到triple.txt中
重新建造碎片（步骤5-7）
mention2ent记得添加

2.调用java服务，reload碎片，添加mention2ent碎片

优化一下关系抽取，提前加载好模型（主要是关系抽取模型）
还有时间的话，把men2entity解决一下
整合上线
做ppt排练

In [113]:
spo_list=[{"predicate": "作者", "object_type": "人物", "subject_type": "图书作品", "object": "祝晓羽", "subject": "九十人飞行组"}]
from py2neo import Graph,Node,Relationship
import json
import os
import requests
from pandas import DataFrame

graph = Graph("bolt://localhost:7687/db/schemaKG/", username="neo4j", password="kuda")

theCypherQuery= '''MERGE (j:%s {name: $subject})
MERGE (m:%s {name: $_object})
MERGE (j)-[r:%s]->(m)
RETURN id(j),type(r),id(m)'''

nodeQuery ="""
match (n) where n.name=$name return n
"""

relationQuery ="""
match (a)-[r]->(b) where a.name=$a_name and b.name=$b_name return r
"""

id_file_path ="/home/lx/IdeaProject/gAnswer/data/pkubase/fragments/id_mappings/pkubase_entity_id.txt"
triple_file_path = "/home/lx/IdeaProject/gAnswer/data/pkubase/triple.txt"

def check_if_node_exist(name):
    res = graph.run(nodeQuery,parameters = {'name':name}).data()
    if len(res) > 0:
        return True
    else:
        return False
    
def check_if_rel_exist(a_name,b_name):
    res = graph.run(relationQuery,parameters = {'a_name':a_name,'b_name':b_name}).data()
    if len(res) > 0:
        return True
    else:
        return False

def insert(thisCypherQuery,subject_type,subject,object_type,_object,predicate):
    query = thisCypherQuery % (subject_type,object_type,predicate)
    cypherResult = graph.run(query,parameters = {'subject':subject,'_object':_object}).data()
    return cypherResult

def get_predicates():
    data = DataFrame(graph.run("match ()-[r]->() return distinct type(r)").data())
    predicates = []
    for index, row in data.iterrows():
        predicates.append(row.to_list()[0])
    return predicates

def get_labels():
    label_list = []
    labels = DataFrame(graph.run("call db.labels()").data())
    for index, row in labels.iterrows():
        if index == 0:
            continue
        label_list.append(row.to_list()[0])
    return label_list

predicates = get_predicates()
labels = get_labels()
id_file = open(id_file_path,"a+")
triple_file = open(triple_file_path,"a+")
new_entity = ""
for spo in spo_list:
    subject_exist = check_if_node_exist(spo['subject'])
    object_exist = check_if_node_exist(spo['object'])
    triple_exist = check_if_rel_exist(spo['subject'],spo['object'])
    
    res = insert(theCypherQuery,spo['subject_type'],spo['subject'],spo['object_type'],spo['object'],spo['predicate'])
    data = DataFrame(res)

    for index,row in data.iterrows():
        
        
        if not subject_exist:
            new_entity+=spo['subject']+","
            if spo['subject_type']=='Date' or spo['subject_type']=='Number' or spo['subject_type']=='Text':
                triple_file.write(str(row['id(j)']) + ' ' + '48 -1\n')
            else:
                triple_file.write(str(row['id(j)'])+' '+'48 '+str(labels.index(spo['subject_type']))+'\n')
            id_file.write('<'+spo['subject']+'>\t'+str(row['id(j)'])+'\n')
            
            
        if not object_exist:
            new_entity+=spo['object']+","
            id_file.write('<'+spo['object']+'>\t'+str(row['id(m)'])+'\n')
            if spo['object_type']=='Date' or spo['object_type']=='Number' or spo['object_type']=='Text':
                triple_file.write(str(row['id(m)']) + ' ' + '48 -1\n')
            else:
                triple_file.write(str(row['id(m)'])+' '+'48 '+str(labels.index(spo['object_type']))+'\n')
            
            
        if not triple_exist:
            triple_file.write(str(row['id(j)'])+' '+str(predicates.index(row['type(r)']))+' '+str(row['id(m)'])+'\n')
            
id_file.close()
triple_file.close()
new_entity = new_entity[:-1]
os.system("python /home/lx/PycharmProjects/kuda2.0/genrate_fragments/step5_get_entity_fragment.py")
os.system("python /home/lx/PycharmProjects/kuda2.0/genrate_fragments/step6_get_type_fragment.py")
os.system("python /home/lx/PycharmProjects/kuda2.0/genrate_fragments/step7_get_predicate_fragment.py")
requests.post('http://localhost:8082/gReload/',data = new_entity.encode('utf-8'))

<Response [200]>

In [56]:
print(labels)

['图书作品', '人物', '机构', 'Text', '影视作品', '娱乐人物', 'Date', '历史人物', '国家', '电视综艺', '学校', '行政区', '气候', '企业', '奖项', '文学作品', '歌曲', '音乐专辑', 'Number', '地点', '景点', '城市', '作品', '企业/品牌', '语言', '学科专业']


In [114]:
import requests
import json
cypher_res = requests.get('http://localhost:8082/gSolve/?data={question:"《九十人飞行组》的作者是谁？"}')
obj = json.loads(cypher_res.text)
print(obj['sparql'])

match (a)-[r]-(b) where a.name='九十人飞行组' and type(r)='作者' return distinct b


In [95]:
import requests
data = "西班牙,墨西哥"

requests.post('http://localhost:8082/gReload/',data = data.encode('utf-8'))

<Response [200]>